In [17]:
import requests     
import numpy as np   
import pandas as pd  
import time        
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import docx2txt
from pathlib import Path
import html5lib
import re
import matplotlib.pyplot as plt
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
import time
import os

**Проблемы парсинга Росстата:**

-Страницы динамические, простой парсинг не работает, я буду использовать Селениум

-Последние года(с 2015 года) инфа представлена в формате .doc а не html таблиц, я буду переносить таблице из .doc в .xlsx а затем импортить пандасом

-Страницы могут иметь разный дизайн, необходим более сложный код или несколько функций

-Дата представлена в разных форматах: сначала идет суммарное потребление алкоголя по регионам, затем подушевое потребление, затем снова суммарное, придется вытаскивать численность населения и находить подушевое потребление

**1.** Найдем все страницы данных по годам

In [3]:
page_link = 'https://rosstat.gov.ru/folder/210/document/13204'
response = requests.get(page_link)
soup = BeautifulSoup(response.content, 'html')

In [4]:
links = soup.find_all('a', class_ = "btn btn-icon btn-white btn-br btn-sm")
pages = []
link = ['https://rosstat.gov.ru/bgd/regl/','/Iss.htm']
for item in links:
    if item.get('href').startswith('http'): pages.append(link[0]+item.get('href').split('/')[5]+link[1])

In [5]:
pages

['https://rosstat.gov.ru/bgd/regl/b21_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/b20_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B19_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B18_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B17_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B16_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B15_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B14_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B13_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B12_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B11_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B10_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B09_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B08_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B07_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B06_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B05_14p/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B04_14/Iss.htm',
 'https://rosstat.gov.ru/bgd/regl/B03_14/Iss.htm']

**2.** Вытащим потребление алкоголя по годам

In [93]:
def take_page(url, first_path, second_path, wording='алко'):##находим ссылку на html таблицу
    driver = webdriver.Chrome() 
    driver.get(url) 
    time.sleep(2)
    try:
        element = driver.find_element('xpath', "//*[(text()='"+first_path[0]+"')]")
    except:
        element = driver.find_element('xpath', "//*[(text()='"+first_path[1]+"')]")
    element.click()
    time.sleep(2)
    try:
        element_1 = driver.find_element('xpath', "//*[text()='"+second_path[0]+"']")
    except:
        element_1 = driver.find_element('xpath', "//*[(text()='"+second_path[1]+"')]")
    element_1.click()
    time.sleep(2)
    elements = driver.find_elements('xpath', "//*[@href]")
    res = dict()
    for el in elements:
        res[el.get_attribute('text')] = el.get_attribute('href')
    driver.quit()
    return(list(value for key, value in res.items() if wording in key.lower()))

In [7]:
def make_df(url):
    response_temp = requests.get(url) 
    soup_temp = BeautifulSoup(response_temp.content,'html')
    tables = soup_temp.find_all('table')
    
    
    df = pd.read_html(str(tables), flavor='bs4')[0]
    years = list(df[df.columns[1:3]].iloc[1])
    df1 = df[df.columns[::2]].drop(1, axis=0)
    df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0])
    df1 = df1.rename(columns={np.nan:'Регион'})
    df1[df1.columns[1::]] = df1[df1.columns[1::]].map(lambda x: float(x) if x.isdigit() else 0).map(lambda x: x/10)
    df1['Год'] = years[1]
    df2 = df[df.columns[0]].drop(1, axis=0)
    df2.iloc[0]='Регион'
    a = df[df.columns[1::2]].drop(1, axis=0)
    df2 = pd.merge(df2, a, left_index=True, right_index=True)
    df2 = df2.rename(columns=df2.iloc[0]).drop(df2.index[0])
    df2[df2.columns[1::]] = df2[df2.columns[1::]].map(lambda x: float(x) if x.isdigit() else 0).map(lambda x: x/10)
    df2['Год'] = years[0]
    return df1, df2

Собираем данные за 2001-2005 в отдельную таблицу, тут суммарное потребление алкоголя по регионам, все данные будут экспортироваться в .csv, далее будут собираться в один датафрейм

In [8]:
trade = ['Торговля и услуги населению', 'ТОРГОВЛЯ И УСЛУГИ НАСЕЛЕНИЮ']
retail = ['Розничная торговля', 'РОЗНИЧНАЯ ТОРГОВЛЯ']
url_ = take_page('https://rosstat.gov.ru/bgd/regl/B03_14/ISS.htm', trade, retail)

In [9]:
df2, df1 = make_df(url_[0])
df = df1._append(df2, ignore_index='True')
for page in pages[17:14:-1]:
    url = take_page(page, trade, retail)[0]
    df2, df1 = make_df(url)
    df2.columns=df.columns
    df = df._append(df2, ignore_index='True')
cols = df.columns[:6]
filepath=Path('Data/2001-2005.csv')
df.to_csv(filepath)##Сохраним в

C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\558925790.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\558925790.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\558925790.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\558925790.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be re

In [10]:
df

,Регион,Водка и ликеро- водочные изделия,Виноградные и плодовые вина,Шампанские и игристые вина,Коньяки,Пиво,Год
0,"Российская Федеpация, млрд. дкл",209.1,56.1,18.5,4.5,634.6,2001
1,Центpальный федеральный округ,59460.5,14305.6,6626.5,1279.5,182444.1,2001
2,Белгородская область,1428.5,511.1,123.6,18.8,3499.2,2001
3,Бpянская область,1120.2,382.3,60.9,9.7,5827.6,2001
4,Владимиpская область,2366.5,366.5,96.7,22.1,4412.4,2001
...,...,...,...,...,...,...,...
435,Камчатская область,776.9,291.9,47.9,71.5,1717.7,2005
436,Магаданская область,283.6,115.0,11.9,35.0,692.0,2005
437,Сахалинская область,1004.3,364.8,53.4,117.8,2077.7,2005
438,Еврейская автономная область,130.0,67.6,2.2,7.2,1086.3,2005


2006-2013 года, тут уже подушевое потребление, сохраним отдельно

In [12]:
def take_df(url, year):
    response_temp = requests.get(url) 
    soup_temp = BeautifulSoup(response_temp.content,'html')
    tables = soup_temp.find_all('table')
    
    df = pd.read_html(str(tables), flavor='bs4')[0]
    df1 = df[df.columns[::2]]
    df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0])
    df1 = df1.rename(columns={np.nan:'Регион'})
    df1['Год'] = year
    df1[df1.columns[1:-1]] = df1[df1.columns[1:-1]].map(lambda x: float(x) if str(x).isdigit() else 0).map(lambda x: x/10)
    return df1

Ниже могут быть ConnectionTimeout, однако данный код уже прогонялся в Черновой тетрадке, данные уже собраны

In [14]:
y = 2006
for page in pages[14:6:-1]:
    url = take_page(page, trade, retail)[0]
    
    df = take_df(url, y)
    filepath = Path('Data/{}.csv'.format(y))
    df.to_csv(filepath)
    y+=1

ConnectTimeout: HTTPSConnectionPool(host='rosstat.gov.ru', port=443): Max retries exceeded with url: /bgd/regl/B07_14p/IssWWW.exe/Stg/d03/20-10.htm (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029CFBA24350>, 'Connection to rosstat.gov.ru timed out. (connect timeout=None)'))

In [15]:
def clear_data(path_in, cols):
    path = Path(f"Data/{path_in}")
    df = pd.read_csv(path)
    df[df.columns[2:-1]] = df[df.columns[2:-1]].loc[~(df[df.columns[2:-1]]==0).all(axis=1)]
    df = df.drop(columns=df.columns[0])
    df_=pd.DataFrame({'A': []})
    for jtem in cols:
        for item in df.columns:
            if jtem.startswith(item[0:3]):
                df_[jtem] = df[item]
    df_ = df_.drop(columns='A')
    df_ = df_.dropna()
    return df_

In [33]:
ready_files = os.listdir('Data')
ready = [x for x in ready_files if x.endswith('.csv')][1::]

In [36]:
ready

['2006.csv',
 '2007.csv',
 '2008.csv',
 '2009.csv',
 '2010.csv',
 '2011.csv',
 '2012.csv',
 '2013.csv']

In [37]:
df = clear_data(ready[0], cols)
for item in ready:
    df = df._append(clear_data(item, cols))
filepath = Path('Data/by_soul/2006-2013.csv')
df.to_csv(filepath)

2014-2019 года

In [38]:
path= Path("Data/2014-2019.xlsx")
df = pd.read_excel(path)
df = df.rename(columns={'Column1':'Регион'}).drop(df.index[0])

In [47]:
y = 2014
df_out = pd.DataFrame(columns=cols)
for i in range(1, len(df.columns), 5):
    df_temp = pd.DataFrame(columns=cols)
    df_temp['Регион'] = df['Регион']
    for item in df.columns[i:i+5]:
        for jtem in cols:
            if item.startswith(jtem[:4]):
                df_temp[jtem] = df[item]
    df_temp['Год']=y
    y+=1
    df_out = df_out._append(df_temp)
df_out = df_out.astype({'Год':'int32'})
path = Path('Data/by_soul/2014-2019.csv')
df_out.to_csv(path)

C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\2294441861.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_out = df_out._append(df_temp)


In [48]:
df_out

,Регион,Водка и ликеро- водочные изделия,Виноградные и плодовые вина,Шампанские и игристые вина,Коньяки,Пиво,Год
1,Российская Федерация,7.700000,6.200000,1.800000,0.800000,61.300000,2014
2,Центpальный федеральный округ,10.100000,7.700000,2.700000,1.100000,72.000000,2014
3,Белгородская область,5.200000,3.000000,1.200000,0.300000,55.500000,2014
4,Бpянская область,6.400000,8.500000,1.400000,0.500000,46.900000,2014
5,Владимиpская область,7.900000,9.600000,1.000000,0.700000,51.400000,2014
...,...,...,...,...,...,...,...
86,Сахалинская область,7.614527,7.623182,2.910284,1.374499,56.952197,2019
87,Еврейская автономная область,5.529455,5.498925,1.462586,0.559361,62.089225,2019
88,Чукотский автономный округ,3.999525,4.175170,1.634483,1.201382,34.399547,2019
89,Республика Крым,4.577286,4.732818,1.773212,0.748570,36.447218,2019


In [50]:
path= Path("Data/2020-2022_raw.xlsx")
df = pd.read_excel(path)
y = 2020
df_out = pd.DataFrame(columns=cols)
for i in range(1, 4):
    df_temp = pd.DataFrame(columns=cols)
    df_temp['Регион'] = df['Unnamed: 0']
    for item in df.columns[i::3]:
        for jtem in cols:
            if item.startswith(jtem[:4]):
                df_temp[jtem] = df[item]
    df_temp['Год']=y
    y+=1
    df_out = df_out._append(df_temp)
df_out = df_out.astype({'Год':'int32'})
path = Path('Data/raw/2020-2022.csv')
df_out.drop(df.index[0], axis=0).to_csv(path)

**3.** Вытащим Численность населения для 2001-2006 и 2019-2022 для последующего получения подушевого потребления для 2002-2006 и 2020-2022 годов

In [51]:
def take_page_less(url, first_path, wording='численность'):
    driver = webdriver.Chrome() 
    driver.get(url) 
    time.sleep(2)
    try:
        element = driver.find_element('xpath', "//*[(text()='"+first_path[0]+"')]")
    except:
        element = driver.find_element('xpath', "//*[(text()='"+first_path[1]+"')]")
        print(element)
    element.click()
    time.sleep(2)
    elements = driver.find_elements('xpath', "//*[@href]")
    res = dict()
    for el in elements:
        res[el.get_attribute('text')] = el.get_attribute('href')
    driver.quit()
    return(list(value for key, value in res.items() if wording in key.lower()))

In [53]:
people = ['Население', 'НАСЕЛЕНИЕ']
quant = ['Предварительные итоги Всероссийской переписи населения','ПРЕДВАРИТЕЛЬНЫЕ ИТОГИ ВСЕРОССИЙСКОЙ ПЕРЕПИСИ НАСЕЛЕНИЯ']

In [54]:
y = 2020
people_number = dict()
for page in pages:
    people_number[y]=take_page_less(page, people, 'численность населения')
    y+=(-1)
people_number

<selenium.webdriver.remote.webelement.WebElement (session="7432c5ec4277ddd1eb329ef0520c90c3", element="f.08CA2A96479BB2D80375AA22518913B9.d.0426DE9144EE753425EBE73430E635C7.e.32")>
<selenium.webdriver.remote.webelement.WebElement (session="ebba79197812ae3dbeca33d8fba69329", element="f.0CACAAA6B7C5B0F6C473A83AD15D72F7.d.59414797FB6FBFE7FD55395650533B16.e.32")>


{2020: ['https://rosstat.gov.ru/bgd/regl/b21_14p/IssWWW.exe/Stg/d1/02-01.docx',
  'https://rosstat.gov.ru/bgd/regl/b21_14p/IssWWW.exe/Stg/d1/02-02.docx'],
 2019: ['https://rosstat.gov.ru/bgd/regl/b20_14p/IssWWW.exe/Stg/d01/02-01.docx',
  'https://rosstat.gov.ru/bgd/regl/b20_14p/IssWWW.exe/Stg/d01/02-02.docx'],
 2018: ['https://rosstat.gov.ru/bgd/regl/B19_14p/IssWWW.exe/Stg/d01/02-01.docx',
  'https://rosstat.gov.ru/bgd/regl/B19_14p/IssWWW.exe/Stg/d01/02-02.docx'],
 2017: ['https://rosstat.gov.ru/bgd/regl/B18_14p/IssWWW.exe/Stg/d01/02-01.doc',
  'https://rosstat.gov.ru/bgd/regl/B18_14p/IssWWW.exe/Stg/d01/02-02.doc'],
 2016: ['https://rosstat.gov.ru/bgd/regl/B17_14p/IssWWW.exe/Stg/d01/02-01.doc',
  'https://rosstat.gov.ru/bgd/regl/B17_14p/IssWWW.exe/Stg/d01/02-02.doc'],
 2015: ['https://rosstat.gov.ru/bgd/regl/B16_14p/IssWWW.exe/Stg/d01/02-01.doc',
  'https://rosstat.gov.ru/bgd/regl/B16_14p/IssWWW.exe/Stg/d01/02-02.doc'],
 2014: ['https://rosstat.gov.ru/bgd/regl/B15_14p/IssWWW.exe/Stg/d0

In [68]:
def take_quant(url, year):
    response_temp = requests.get(url) 
    soup_temp = BeautifulSoup(response_temp.content,'html')
    tables = soup_temp.find_all('table')
    
    df = pd.read_html(str(tables), flavor='bs4')[0]
    df = df[df.columns[0::len(df.columns)-1]].drop(df.index[0], axis=0)
    df = df.rename(columns={df.columns[0]:'Регион', df.columns[1]:'Численность'})
    df['Год'] = year
    return df

In [79]:
response_temp = requests.get('https://rosstat.gov.ru/bgd/regl/B03_14/IssWWW.exe/Stg/d010/i010090r.htm') 
soup_temp = BeautifulSoup(response_temp.content,'html')
tables = soup_temp.find_all('table')
    
df_q = pd.read_html(str(tables), flavor='bs4')[0]
df_q = df_q[df_q.columns[0:5:4]].drop(df_q.index[0], axis=0)
df_q = df_q.rename(columns={df_q.columns[0]:'Регион', df_q.columns[1]:'Численность'})
df_q['Год'] = 2002
df_q = df_q.dropna()
df_q = pd.DataFrame(columns=['Регион', 'Численность', 'Год'])
for i in range(2003, 2007):
    df_q = df_q._append(take_quant(people_number[i][0], i).dropna())

C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\21668094.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_q = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\1916188061.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\1916188061.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\1916188061.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will b

In [83]:
path = Path('Data/quant/2002-2006.csv')
df_q.to_csv(path)

**4.** Получим реальные зарплаты

In [99]:
stand_of_living = ['УРОВЕНЬ ЖИЗНИ НАСЕЛЕНИЯ','Уровень жизни населения']
wealth = ['ДЕНЕЖНЫЕ ДОХОДЫ НАСЕЛЕНИЯ', 'Денежные доходы населения']
wording = 'реаль'

In [100]:
y = 2020
people_salary = dict()
for page in pages:
    people_salary[y]=take_page(page, stand_of_living, wealth, wording)
    y-=1

In [88]:
def take_wealth(url, year, space, wording):
    response_temp = requests.get(url) 
    soup_temp = BeautifulSoup(response_temp.content,'html')
    tables = soup_temp.find_all('table')
    
    df = pd.read_html(str(tables), flavor='bs4')[0]
    df = df[df.columns[0:space+1:space]].drop(df.index[0], axis=0)
    df = df.rename(columns={df.columns[0]:'Регион', df.columns[1]:wording})
    df['Реальные денежные доходы населения'] = pd.to_numeric(df['Реальные денежные доходы населения'], errors='coerce')/10
    df['Год'] = year
    return df

In [102]:
y = 2002
wording = 'Реальные денежные доходы населения'
df_q = pd.DataFrame(columns=['Регион', 'Реальные денежные доходы населения', 'Год'])
for i in range(y, 2014):
    df_q = df_q._append(take_wealth(people_salary[i][0], i, 5, wording))

C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\145990495.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0]
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\2774814643.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_q = df_q._append(take_wealth(people_salary[i][0], i, 5, wording))
C:\Users\maxkh\AppData\Local\Temp\ipykernel_22172\145990495.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables), flavor='bs4')[0

In [105]:
path = Path('Data/wealth/2002-2013.csv')
df_q.dropna().to_csv(path)